# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace
/home/workspace/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

#print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect') 
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster()
    # establish connection and begin executing queries
    session = cluster.connect()
    print("Cluster connected!")
except Exception as e:
    print(e)

Cluster connected!


#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION =
   { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""               
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



## Query 1
### The request is run a query on sessionId and ItemInSessionId where sessionId = 338 and itemInSession = 4
### The Primary key are sessionId and ItemInSessionId, wehere sessionId is the partition key and itemInSession is the clustering column 
### Cloumn included : artist, song title and song's length

In [18]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \

create_table1 = 'CREATE TABLE IF NOT EXISTS music_session (sessionId int, itemInsession int, artist text, song_title text, song_length float, \
                PRIMARY KEY(sessionId, itemInsession))'

try:
    session.execute(create_table1)
    print("Table created!")
except Exception as e:
    print(e)

                    

Table created!


In [19]:
## Read from the event file
file = 'event_datafile_new.csv'

# read from csv file, insert data into database
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)   
    next(csvreader) 
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_session(sessionId, itemInsession, artist, song_title, song_length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        query_data = (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))

        try:
            session.execute(query, query_data)
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [20]:
## SELECT the data 
query = 'SELECT artist, song_title, song_length FROM music_session WHERE sessionId = 338 AND  itemInsession = 4'
try:
    result_set = session.execute(query)
except Exceptiopn as e:
    print(e)

for result in result_set:
    print(result)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


## Query 2  
### The request is run a query on userId and sessionId
### Cloumn included : name of artist, song, user first name and user last name
### Primary Key is ((sessionId, userId), itemInsession). SessionId and userId is the partition key and itemInsession is the clustering column
### CLUSTERING ORDER BY will be used to sort data based on itemInsession

In [21]:
## Create table for query2
create_table2 = 'CREATE TABLE IF NOT EXISTS user_session (artist text, song_title text, itemInsession int, \
                 user_first_name text, user_last_name text, userId int, sessionId int, PRIMARY KEY((userId, sessionId),iteminsession))\
                 WITH ClUSTERING ORDER BY (itemInsession ASC)'

try:
    session.execute(create_table2)
    print("Table created!")
except Exception as e:
    print(e)
                    

Table created!


In [22]:
# Insert data
file = 'event_datafile_new.csv'

with open(file, encoding ='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_session(artist, song_title, itemInsession, user_first_name, \
                user_last_name, userId, sessionId)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        query_data = (line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8]))
        
        try:
            session.execute(query, query_data)
        except Exception as e:
            print(e)

In [23]:
# Confirm data insertion
query2 = 'SELECT artist, song_title, user_first_name, user_last_name FROM user_session \
         WHERE userId = 10 AND sessionId = 182 '
result_set = session.execute(query2)

for result in result_set:
    print(result)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


## Query 3
### The request is run a query on run a query on song
### Cloumn included : Select song, user firstname, user lastname, userId From user_song_library
### Primary Key is (song, userId). Song is the partition key and userId is the clustering column

In [24]:
## Create table for query3
create_query3 = """CREATE TABLE IF NOT EXISTS song_user (song_title text, userId int, user_firstName text, user_lastName text, 
                PRIMARY KEY ((song_title), userId))"""

try: 
    session.execute(create_query3)
    print("Table created!")
except Exception as e:
    print(e)
                    

Table created!


In [25]:
## Insert date
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_user (song_title, userId, user_firstName, user_lastName) "
        query = query + "VALUES (%s, %s, %s, %s)"
        query_data = ( line[9], int(line[10]), line[1], line[4] )
    
        # load data into table
        try:
            session.execute(query, query_data)
        except Exception as e:
            print(e)


In [26]:
## Confirm data insertion
select_query = "SELECT song_title, user_firstName, user_lastName FROM song_user \
               WHERE song_title = 'All Hands Against His Own'"
try:
    result_set = session.execute(select_query)
except Exception as e:
    print(e)

for result in result_set:
    print(result)
    

Row(song_title='All Hands Against His Own', user_firstname='Jacqueline', user_lastname='Lynch')
Row(song_title='All Hands Against His Own', user_firstname='Tegan', user_lastname='Levine')
Row(song_title='All Hands Against His Own', user_firstname='Sara', user_lastname='Johnson')


### Drop the tables before closing out the sessions

In [27]:
# Drop the table before closing out the sessions
query1 = "DROP TABLE song_user"
query2 = "DROP TABLE user_session"
query3 = "DROP TABLE music_session"

query_list = [query1, query2, query3]

for query in query_list:
    session.execute(query)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()